In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import acquire

In [9]:
zillow= acquire.get_zillow_data()
zillow=zillow.drop(columns='Unnamed: 0')

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
zillow.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,...,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,1727539,NaN,3.5,...,6.059063e+13,0,0.025595,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,1387261,NaN,1.0,...,6.111001e+13,1,0.055619,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,11677,NaN,2.0,...,6.059022e+13,2,0.005383,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,NaN,NaN,NaN,2288172,NaN,3.0,...,6.037300e+13,3,-0.103410,2017-01-01,NaN,NaN,NaN,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,NaN,NaN,1.0,1970746,NaN,3.0,...,6.037124e+13,4,0.006940,2017-01-01,Central,NaN,NaN,Condominium,NaN,NaN


In [11]:
zillow.shape

(77613, 68)

In [12]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77613 entries, 0 to 77612
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77613 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      207 non-null    float64
 6   airconditioningtypeid         25007 non-null  float64
 7   id                            77613 non-null  int64  
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77579 non-null  float64
 10  bedroomcnt                    77579 non-null  float64
 11  buildingqualitytypeid         49809 non-null  float64
 12  calculatedbathnbr             76963 non-null  float64
 13  d

#### Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [13]:
def table_information(df):
    
    '''This function will read in a dataframe and returns a new dataframe with the nuber of columns missing, percent of columns missing, and number of rows with n columns missing. '''
    num_rows = df.loc[:].isnull().sum()
    num_cols_missing = df.loc[:, df.isna().any()].count()
    pct_cols_missing = round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)
    missing_cols_and_rows_df = pd.DataFrame({'number_of_columns_missing': num_cols_missing,
                                             'percent_of_columns_missing': pct_cols_missing,
                                             'number_of_rows': num_rows})
    missing_cols_and_rows_df = missing_cols_and_rows_df.fillna(0)
    missing_cols_and_rows_df['number_of_columns_missing'] = missing_cols_and_rows_df['number_of_columns_missing'].astype(int)
    return missing_cols_and_rows_df

In [14]:
table_information(zillow)

,number_of_columns_missing,percent_of_columns_missing,number_of_rows
airconditioningdesc,25007,32.220,52606
airconditioningtypeid,25007,32.220,52606
architecturalstyledesc,207,0.267,77406
architecturalstyletypeid,207,0.267,77406
assessmentyear,77579,99.956,34
...,...,...,...
typeconstructiontypeid,223,0.287,77390
unitcnt,50703,65.328,26910
yardbuildingsqft17,2393,3.083,75220
yardbuildingsqft26,70,0.090,77543


#### Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [19]:
unit_df = zillow[zillow['unitcnt']==1]
room_df = zillow[zillow['roomcnt']>0]
garage_df = zillow[zillow['garagecarcnt']>0]
bed_df = zillow[zillow['bedroomcnt']>0]
bath_df = zillow[zillow['bathroomcnt']>0]
p261_df = zillow[zillow['propertylandusetypeid'] == 261]
p263_df = zillow[zillow['propertylandusetypeid'] == 263]
p264_df = zillow[zillow['propertylandusetypeid'] == 264]    
p266_df = zillow[zillow['propertylandusetypeid'] == 266] 
p273_df = zillow[zillow['propertylandusetypeid'] == 273]
p275_df = zillow[zillow['propertylandusetypeid'] == 275]
p276_df = zillow[zillow['propertylandusetypeid'] == 276]    
p279_df = zillow[zillow['propertylandusetypeid'] == 279]
single_unit_df = pd.concat([unit_df, room_df, garage_df,bed_df,bath_df,p261_df, p263_df, p264_df, p266_df, p273_df, p275_df, p276_df, p279_df]).drop_duplicates('id').reset_index(drop=True)
single_unit_df.shape

(77056, 68)

In [20]:
def remove_non_single_unit_props (zillow):
    unit_df = zillow[zillow['unitcnt']==1]
    room_df = zillow[zillow['roomcnt']>0]
    garage_df = zillow[zillow['garagecarcnt']>0]
    bed_df = zillow[zillow['bedroomcnt']>0]
    bath_df = zillow[zillow['bathroomcnt']>0]
    p261_df = zillow[zillow['propertylandusetypeid'] == 261]
    p263_df = zillow[zillow['propertylandusetypeid'] == 263]
    p264_df = zillow[zillow['propertylandusetypeid'] == 264]    
    p266_df = zillow[zillow['propertylandusetypeid'] == 266] 
    p273_df = zillow[zillow['propertylandusetypeid'] == 273]
    p275_df = zillow[zillow['propertylandusetypeid'] == 275]
    p276_df = zillow[zillow['propertylandusetypeid'] == 276]    
    p279_df = zillow[zillow['propertylandusetypeid'] == 279]
    single_unit_df = pd.concat([unit_df, room_df, garage_df,bed_df,bath_df,p261_df, p263_df, p264_df, p266_df, p273_df, p275_df, p276_df, p279_df]).drop_duplicates('id').reset_index(drop=True)
    return single_unit_df

In [24]:
single_unit_df= remove_non_single_unit_props(zillow)

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
<ol>
<li>A dataframe
<li>A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
<li>A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
    </ol>

The output:
<ol>
<li>The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
    </ol>
hint:
<ul>
<li> Look up the dropna documentation.
<li>You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
<li>Make use of inplace, i.e. inplace=True/False.
<li>Decide how to handle the remaining missing values:
</ul>
Fill with constant value.
Impute with mean, median, mode.
Drop row/column

In [ ]:
def prep_zillow (df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    def remove_columns(df, cols_to_remove):  
        df = df.drop(columns=cols_to_remove)
        return df
    def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
        threshold = int(round(prop_required_column*len(df.index),0))
        df.dropna(axis=1, thresh=threshold, inplace=True)
        threshold = int(round(prop_required_row*len(df.columns),0))
        df.dropna(axis=0, thresh=threshold, inplace=True)
        return df
    df = remove_columns(df, cols_to_remove)  # Removes Specified Columns
    df = handle_missing_values(df, prop_required_column, prop_required_row) # Removes Specified Rows
    #df.dropna(inplace=True) # Drops all Null Values From Dataframe
    return df

In [25]:
prep_zillow(single_unit_df,
    cols_to_remove=[],
    prop_required_column=.6,
    prop_required_row=.75 )    

,parcelid,propertylandusetypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate,propertylandusedesc
0,12177905,261.0,2288172,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,3,-0.103410,2017-01-01,Single Family Residential
1,10887214,266.0,1970746,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,4,0.006940,2017-01-01,Condominium
2,12095076,261.0,781532,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,6,-0.001011,2017-01-01,Single Family Residential
3,12069064,261.0,870991,1.0,2.0,5.0,1.0,738.0,738.0,6037.0,...,18890.0,218552.0,2016.0,199662.0,2366.08,6.037302e+13,7,0.101723,2017-01-01,Single Family Residential
4,12790562,261.0,1246926,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,8,-0.040966,2017-01-02,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77046,17190561,275.0,791115,0.0,0.0,NaN,NaN,1536.0,NaN,6111.0,...,NaN,43264.0,2016.0,43264.0,443.20,6.111006e+13,53718,-0.017181,2017-07-03,"Manufactured, Modular, Prefabricated Homes"
77048,17190562,275.0,1357890,0.0,0.0,NaN,NaN,1440.0,NaN,6111.0,...,NaN,180000.0,2016.0,180000.0,1970.70,6.111006e+13,54582,-0.005354,2017-07-06,"Manufactured, Modular, Prefabricated Homes"
77050,17190635,275.0,970220,0.0,0.0,NaN,NaN,1440.0,NaN,6111.0,...,NaN,180000.0,2016.0,180000.0,1896.30,6.111006e+13,58827,0.080876,2017-07-20,"Manufactured, Modular, Prefabricated Homes"
77051,17190478,275.0,2029177,0.0,0.0,NaN,NaN,1440.0,NaN,6111.0,...,NaN,99309.0,2016.0,99309.0,1038.80,6.111006e+13,66627,0.060742,2017-08-15,"Manufactured, Modular, Prefabricated Homes"
